# Pendant drop measurement check

In [ ]:
# Imports

## Packages

## Custom code
from hardware.opentrons.opentrons_api import OpentronsAPI
from hardware.opentrons.configuration import Configuration
from hardware.opentrons.droplet_manager import DropletManager
from hardware.cameras.pendant_drop_camera import PendantDropCamera
from analysis.plots import Plotter
from utils.load_save_functions import load_settings
from utils.utils import calculate_equillibrium_value
from hardware.sensor.sensor_api import SensorAPI

# Initialisation
opentrons_api = OpentronsAPI()
opentrons_api.initialise()
config = Configuration(opentrons_api=opentrons_api)
labware = config.load_labware()
containers = config.load_containers()
pipettes = config.load_pipettes()
left_pipette = pipettes["left"]
right_pipette = pipettes["right"]
pendant_drop_camera = PendantDropCamera() 
sensor = SensorAPI()
plotter = Plotter()
droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter
)
settings = load_settings()

In [ ]:
opentrons_api.home()

## Part 0: Set well of interest

In [ ]:
well_ID_of_interest = "7H1"
droplet_manager.source = containers[well_ID_of_interest]

## Part 1: Focus camera

### Prepare pendant drop via standard method

In [ ]:
droplet_manager._prepare_pendant_drop()

### Dispense pendant drop manually
repeat this block to dispense more

In [ ]:
left_pipette.dispense(
    volume=0,
    destination=containers['drop_stage'],
    depth_offset=settings["PENDANT_DROP_DEPTH_OFFSET"],
    flow_rate=1,
    log=False,
    update_info=False
)

### Steps to focus camera
- go to [`check_camera.py`](check_camera.py)
- run script, a video stream of the pendant drop should occur
- manually focus the camera
- press q to exit

### Return pendant drop

In [ ]:
volume_dispensed = 17 - left_pipette.volume # prepare pendant drop aspirate 17 uL of source (p20 - 3 uL air gap)
droplet_manager._return_pendant_drop(
    drop_volume=volume_dispensed
)

## Part 2: Check measurement

In [ ]:
well_ID_of_interest = "7B2"

In [ ]:
dynamic_surface_tension, drop_volume, drop_count = droplet_manager.measure_pendant_drop(
    source=containers[well_ID_of_interest],
    max_measure_time=30 #s
)
st_eq = calculate_equillibrium_value(
    x=dynamic_surface_tension,
    n_eq_points=100,
    column_index=1,
)

print(f"equillibrium st: {st_eq} mN/m")
meta_data = sensor.capture_sensor_data()
print(f"temperature: {meta_data["Temperature (C)"]}")
print(f"humidity: {meta_data["Humidity (%)"]}")
